In [3]:
from tqdm import tqdm
import numpy as np
from kernels.kernel import Kernel

class LAKernel(Kernel):

    #http://cazencott.info/dotclear/public/publications/Azencott_KernelsForSequences09.pdf
    #https://www.cs.princeton.edu/~bee/courses/read/saigo-bioinformatics-2004.pdf?fbclid=IwAR0LjydQ9kD6FY8ISBsRiYGk_7iopyQAwVKGvzCFoF9EO-O6498av6NcRK4

    def __init__(self):
        super().__init__()

        # gap penalty
        self.d = 1. #gap opening
        self.e = 1. #11. #extension cost

        #against diagonal dominance
        self.beta = 0.5
        
        #csts used in dynamic programming
        self.cst1 = np.exp(self.beta*self.d)
        self.cst2 = np.exp(self.beta*self.e)

        self.type = "BLAST" #or "Transversion" #or ""Identity"

    def score_substitution(self, x, y):
        # substitution table
        # https://en.wikipedia.org/wiki/Substitution_matrix
        # (https://en.wikipedia.org/wiki/Models_of_DNA_evolution)
        # https://slideplayer.com/slide/5092656/

        if self.type == "BLAST":
            equal = int(x==y)
            return 5 * equal - 4 * (1-equal)
        elif self.type == "Transversion":
            equal = int(x == y)
            return 1 * equal - 5 * (1 - equal)
        elif self.type=="Identity":
            equal = int(x == y)
            return equal
        else:
            exit()

    def evaluate(self, x, y):
        """
        Evaluation function computing the inner product between phi_x and phi_y
        """

        #print(x[0],y[0])

        nx = len(x[0])+1
        ny = len(y[0])+1
        M =  np.zeros((nx,ny))
        X =  np.zeros((nx,ny))
        Y =  np.zeros((nx,ny))
        X2 = np.zeros((nx,ny))
        Y2 = np.zeros((nx,ny))
        
        for i in range(1,nx):
            for j in range(1,ny):
                
                #print(cst1,cst2)
                s = self.score_substitution(x[0][i-1],y[0][j-1])
                M[i,j] = np.exp(self.beta*s)*(1+X[i-1][j-1]+Y[i-1][j-1]+M[i-1][j-1])
                X[i,j] = self.cst1*M[i-1,j] + self.cst2*X[i-1,j]
                Y[i,j] = self.cst1*(M[i,j-1] + X[i,j-1]) + self.cst2*Y[i,j-1]
                X2[i,j] = M[i-1,j] + X2[i-1,j]
                Y2[i,j] = M[i,j-1] + X2[i,j-1] + Y2[i,j-1]
                
        #print(1 + X2[-1,-1] + Y2[-1,-1] + M[-1,-1])

        return 1/self.beta * np.log(1 + X2[-1,-1] + Y2[-1,-1] + M[-1,-1])

    def compute_train(self, Xtr):
        print("Computing Train Kernel ...")
        start = time.time()
        n = len(Xtr)
        K = np.zeros((n, n))
        pairs = []
        for i in range(n):
            for j in range(i + 1):
                pairs.append((i,j))
        for (i,j) in tqdm(pairs):
                K[j, i] = self.evaluate(Xtr[i], Xtr[j])
        # Symmetrize Kernel
        K = K + K.T - np.diag(K.diagonal())
        
        eigenv = None #smallest negative eigenvalue
        for e in np.real(np.linalg.eigvals(K)):
            if e<0:
                if eigenv is None:
                    eigenv = e
                else:
                    eigenv = max(eigenv,e)
        if not(eigenv is None):
            K = K - eigenv * np.eye(n)
            
        end = time.time()
        print("Time elapsed: {0:.2f}".format(end - start))

        return K

    def compute_test(self, Xtr, Xte):
        print("Computing Test Kernel ...")
        start = time.time()
        n = len(Xtr)
        m = len(Xte)
        K_t = np.zeros((m, n))
        pairs = []
        for k in range(m):
            for j in range(n):
                pairs.append((k,j))
                
        for (k,j) in tqdm(pairs):
            K_t[k, j] = self.evaluate(Xte[k], Xtr[j])

        #if not(self.smallest_neg_eigenv_train is None):
        #    K_t = K_t - self.smallest_neg_eigenv_train * np.eye(m)[:,:n]

        end = time.time()
        print("Time elapsed: {0:.2f}".format(end - start))
        return K_t


In [4]:
import numpy as np
from cvxopt import matrix, solvers
from kernels.centered_kernel import *

import math
EPS = math.pow(10,-5)

class SVM:
    """
        Implements Support Vector Machine.
    """

    def __init__(self, kernel=None, center=False):
        self.kernel = kernel
        self.center = center

    def init_train(self, Xtr, Ytr, K):

        self.Xtr = Xtr
        self.Ytr = Ytr
        self.n = len(Xtr)

        if self.center:
            print("Centered K")
            if not isinstance(self.kernel, CenteredKernel):
                self.kernel = CenteredKernel(self.kernel)
        if K is None:
            print("Building the Kernel ...")
            self.K = self.kernel.compute_train(self.Xtr)
            print("Kernel built successfully !")
        else:
            self.K = K

    def train(self, Xtr, Ytr, lambd=1, K=None):
        self.init_train(Xtr, Ytr, K)
        print("Solving SVM optimization, please wait ...")
        P = matrix(self.K, tc='d')
        q = matrix(-Ytr, tc='d')
        G = matrix(np.append(np.diag(-Ytr.astype(float)), np.diag(Ytr.astype(float)), axis=0), tc='d')
        h = matrix(np.append(np.zeros(self.n), np.ones(self.n, dtype=float) / (2 * lambd * self.n), axis=0), tc='d')
        solvers.options['show_progress'] = False
        self.alpha = np.array(solvers.qp(P, q, G, h)['x'])
        print("SVM optimization solved !")

        self.alpha = self.alpha.flatten()

        self.idx_SV = np.argwhere(np.abs(self.alpha)>EPS).flatten()
        print("number of SV : ", self.idx_SV.shape[0])
    
        self.Xtr_SV = {i:self.Xtr[self.idx_SV[i]] for i in range(self.idx_SV.shape[0])}
        self.alpha_SV = self.alpha[np.ix_(self.idx_SV)]
        print("alpha values of SV ", self.alpha_SV)

    def score_train(self):
        if self.center:
            f = np.sign(self.K.dot(self.alpha.reshape((self.alpha.size, 1))))
        else:
            K_t = self.K[np.ix_(np.arange(self.n),self.idx_SV)]
            f = np.sign(K_t.dot(self.alpha_SV.reshape((self.alpha_SV.size, 1))))
        return np.mean(f.reshape(-1)==self.Ytr)

    def predict(self, Xte, K_t=None):
        print("Predicting Test sets, please wait ...")
        if self.center:
            if K_t is None:
                self.K_t = self.kernel.compute_test(self.Xtr, Xte)
            else:
                self.K_t = K_t
            Yte = self.K_t.dot(self.alpha.reshape((self.alpha.size, 1))).reshape(-1)
        else:
            if K_t is None:
                self.K_t = self.kernel.compute_test(self.Xtr_SV, Xte)
            else:
                self.K_t = K_t
            Yte = self.K_t.dot(self.alpha_SV.reshape((self.alpha_SV.size, 1))).reshape(-1)
        print("Test predictions computed successfully !")
        return np.sign(Yte)

    def score(self, Xt, Yt):
        predictions = self.predict(Xt, K_t=None)
        #print(predictions,Yt)
        return np.mean(predictions==Yt)

In [5]:
import numpy as np
import pandas as pd

from submission import *
import time
from utils import *


# Read training set 0
Xtr0 = pd.read_csv('./data/Xtr0.csv', sep=',', header=0)
Xtr0 = Xtr0['seq'].values

Ytr0 = pd.read_csv('./data/Ytr0.csv', sep=',', header=0)
Ytr0 = Ytr0['Bound'].values
# Map the 0/1 labels to -1/1
Ytr0 = 2*Ytr0-1

###############################################################################
##############################  TRAIN SESSION #################################
###############################################################################

end_train = 100
end_val = 20
X_tr0_train = Xtr0[:end_train]
Y_tr0_train = Ytr0[:end_train]
X_tr0_val = Xtr0[end_train:end_train+end_val]
Y_tr0_val = Ytr0[end_train:end_train+end_val]

In [6]:
print(">>> Set 0")
Xtr0_merged = {}
for i in range(end_train): #len(Xtr0)
    Xtr0_merged[i] = (X_tr0_train[i],) 
Xtr0 = Xtr0_merged.copy()

Xval0_merged = {}
for i in range(end_val): #len(Xtr0)
    Xval0_merged[i] = (X_tr0_val[i],) 
Xval0 = Xval0_merged.copy()

>>> Set 0


In [7]:
lbd0 = 10.

svm0 = SVM(LAKernel(), center=False)
svm0.train(Xtr0, Y_tr0_train, lbd0)

print("Training accuracy:", svm0.score_train())

  0%|          | 2/5050 [00:00<05:52, 14.31it/s]

Building the Kernel ...
Computing Train Kernel ...


100%|██████████| 5050/5050 [06:04<00:00, 13.89it/s]

Time elapsed: 364.19
Kernel built successfully !
Solving SVM optimization, please wait ...
SVM optimization solved !
number of SV :  94
alpha values of SV  [-1.92418638e-04  4.99999993e-04  4.99999993e-04 -4.99999514e-04
  4.99999993e-04 -4.49869392e-04 -4.62044143e-04 -4.99999364e-04
 -1.36269538e-04 -3.39592713e-04 -4.99996115e-04 -4.99999219e-04
  4.99999993e-04  4.99999993e-04 -3.13226340e-04 -4.99999175e-04
  4.99999993e-04 -4.99978796e-04 -4.99998832e-04  4.99999993e-04
 -2.16685314e-04 -4.98044963e-04 -4.99998221e-04  4.99999993e-04
  4.99999993e-04  4.99999993e-04  4.99999993e-04  4.99999993e-04
  4.99999993e-04 -4.10809968e-04  4.99999993e-04 -1.22475708e-04
 -4.99998598e-04 -4.99997581e-04 -4.99985402e-04 -4.35173824e-04
 -1.24683985e-05  4.99999993e-04  4.99999993e-04  4.99999993e-04
 -2.51624053e-04 -4.99997753e-04 -2.04948836e-04  4.99999993e-04
  4.99999993e-04  4.99999993e-04 -4.98024249e-04 -2.91610799e-04
 -7.88253666e-05  4.99999993e-04 -4.99997826e-04 -4.99998795e-04

In [8]:
print(svm0.alpha_SV)
#print(s)

[-1.92418638e-04  4.99999993e-04  4.99999993e-04 -4.99999514e-04
  4.99999993e-04 -4.49869392e-04 -4.62044143e-04 -4.99999364e-04
 -1.36269538e-04 -3.39592713e-04 -4.99996115e-04 -4.99999219e-04
  4.99999993e-04  4.99999993e-04 -3.13226340e-04 -4.99999175e-04
  4.99999993e-04 -4.99978796e-04 -4.99998832e-04  4.99999993e-04
 -2.16685314e-04 -4.98044963e-04 -4.99998221e-04  4.99999993e-04
  4.99999993e-04  4.99999993e-04  4.99999993e-04  4.99999993e-04
  4.99999993e-04 -4.10809968e-04  4.99999993e-04 -1.22475708e-04
 -4.99998598e-04 -4.99997581e-04 -4.99985402e-04 -4.35173824e-04
 -1.24683985e-05  4.99999993e-04  4.99999993e-04  4.99999993e-04
 -2.51624053e-04 -4.99997753e-04 -2.04948836e-04  4.99999993e-04
  4.99999993e-04  4.99999993e-04 -4.98024249e-04 -2.91610799e-04
 -7.88253666e-05  4.99999993e-04 -4.99997826e-04 -4.99998795e-04
 -4.11300368e-04 -4.99998086e-04 -4.14932576e-04 -1.55455580e-04
 -4.99997673e-04 -4.97391247e-04 -1.42640576e-04  4.99999993e-04
  4.99999993e-04  4.99999

In [9]:
s = svm0.score(Xval0, Y_tr0_val)

  0%|          | 2/1880 [00:00<02:16, 13.79it/s]

Predicting Test sets, please wait ...
Computing Test Kernel ...


100%|██████████| 1880/1880 [02:14<00:00, 13.87it/s]

Time elapsed: 134.56
Test predictions computed successfully !


In [10]:
print(s)

0.55
